In [1]:
from sapphire import *

     Management IP   Password
0    10.127.119.83   nbv_3124
1   10.127.119.103   nbv_3124
2    10.127.119.84   nbv_3124
3    10.127.119.98   nbv_3124
4   10.127.125.228  nbv_12345
5   10.127.125.229  nbv_12345
6   10.127.125.230  nbv_12345
7   10.127.125.239  nbv_12345
8    172.25.169.24  nbv_12345
9   172.25.169.251  nbv_12345
10   172.25.169.12  nbv_12345
11  172.25.169.253  nbv_12345
12  172.25.169.250  nbv_12345
13  172.25.169.239  nbv_12345
{'10.127.119.83': 'nbv_3124', '10.127.119.103': 'nbv_3124', '10.127.119.84': 'nbv_3124', '10.127.119.98': 'nbv_3124', '10.127.125.228': 'nbv_12345', '10.127.125.229': 'nbv_12345', '10.127.125.230': 'nbv_12345', '10.127.125.239': 'nbv_12345', '172.25.169.24': 'nbv_12345', '172.25.169.251': 'nbv_12345', '172.25.169.12': 'nbv_12345', '172.25.169.253': 'nbv_12345', '172.25.169.250': 'nbv_12345', '172.25.169.239': 'nbv_12345'}


In [2]:
mgmt_ip = '10.127.190.136'
password = 'nbv_12345'

mgmt_ip = '10.127.125.228'
password = 'nbv_12345'

mgmt_ip = '10.127.119.98'
password = 'nbv_3124'

mgmt_ip = '172.25.169.24'
password = 'nbv_12345'

switch = Switch(mgmt_ip, password)

Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fccd29d1be0>
Switch object created with paramiko client in-built.


In [3]:
switch

In [4]:
fabric = Fabric(switch)

Fabric object initialized


In [5]:
fabric.search_devices(vsan = 1)

PEER IP = 172.25.169.251 SWITCHNAME = MDS9148S-169251
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fccd29d1940>
Switch object created with paramiko client in-built.
Device succeesfully added to Fabric.
PEER IP = 172.25.169.12 SWITCHNAME = MDS9706-16912
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fccd2a1d6d0>
Switch object created with paramiko client in-built.
Device succeesfully added to Fabric.
PEER IP = 172.25.169.24 SWITCHNAME = N9K-FX-16924
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fccd2a467c0>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 172.25.169.253 SWITCHNAME = MDS9222i-169253
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fccd2a86640>


In [6]:
# See https://neo4j.com/developer/aura-connect-driver/ for Aura specific connection URL.
scheme = "neo4j"  # Connecting to Aura, use the "neo4j+s" URI scheme
host_name = "localhost"
port = 7687
url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
user = "admin"
password = "admin"
app = App(url, user, password)

In [7]:
for switch in fabric.devices:
    switchname = switch.switchname
    print(switch.switchname, switch.mgmt_ip)

    peers = switch.get_topology(vsan = 101)
    print(peers.head())
    for i in range(peers.shape[0]):
        peer_switch = peers['Switch Name'][i]
        app.create_link(switchname, peer_switch, 'E')

    flogis = switch.get_flogi_database(vsan = 101)
    print(flogis.head())
    for i in range(flogis.shape[0]):
        fcid = flogis['FCID'][i]
        app.create_link(switchname, fcid, 'N')

N9K-FX-16924 172.25.169.24
  Interface VSAN      FCID                PORT NAME                NODE NAME
0     fc1/1  101  0xe80000  21:00:00:0e:1e:09:b3:22  20:00:00:0e:1e:09:b3:22
1     fc1/3  101  0xe80d00  21:00:00:0e:1e:09:b3:2d  20:00:00:0e:1e:09:b3:2d
2     fc1/4  101  0xe80100  10:00:00:10:86:04:e0:ad  20:00:00:10:86:04:e0:ad
3     fc1/6  101  0xe80900  21:00:00:24:ff:7e:e6:30  20:00:00:24:ff:7e:e6:30
4     fc1/7  101  0xe80400  21:00:00:0e:1e:09:b3:00  20:00:00:0e:1e:09:b3:00
MDS9148S-169251 172.25.169.251
Empty DataFrame
Columns: [Interface, VSAN, FCID, PORT NAME, NODE NAME]
Index: []
MDS9706-16912 172.25.169.12
  Interface VSAN      FCID                PORT NAME                NODE NAME
0     fc1/3  101  0x190601  20:01:00:a0:98:f4:1e:b2  20:02:00:a0:98:f4:1e:b2
1     fc1/3  101  0x190602  50:0a:09:84:80:c2:f6:f6  50:0a:09:80:80:c2:f6:f6
2     fc2/8  101  0x190560  50:0a:09:86:a0:e0:05:04  50:0a:09:80:80:e0:05:04
3     fc2/8  101  0x190561  20:37:00:a0:98:9f:90:71  20:08:00:a

In [8]:
app.close()